<a href="https://colab.research.google.com/github/gerasimos-matidis/my_icdar/blob/main/segm_w_on_the_fly_augm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd gdrive/MyDrive/my_icdar

/content/gdrive/MyDrive/my_icdar


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
import os
from google.colab import files
from gm_networks import unet_model

In [ ]:
x = plt.imread('train/101-INPUT.jpg')
y = plt.imread('train/101-OUTPUT-GT.png')

x = np.expand_dims(x, 0)
y = np.expand_dims(y, (0, -1))

In [ ]:
NEW_SIZE = 256
DATASET_IMAGES = 30
concatenated_images = np.concatenate([x, y], -1)

croped_images = np.zeros([DATASET_IMAGES, NEW_SIZE, NEW_SIZE, 4])

for i in range(DATASET_IMAGES):
    
    t = tf.image.random_crop(concatenated_images, size=[1, NEW_SIZE, NEW_SIZE, 4])
    croped_images[i] = t[0]

x_train = croped_images[:, :, :, :3]
y_train = np.expand_dims(croped_images[:, :, :, 3], -1)

In [ ]:
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', cval=0)

y_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', cval=0, 
    preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype))

In [ ]:
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

seed = 17
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator = x_datagen.flow(x_train, seed=seed, batch_size=8)
y_generator = y_datagen.flow(y_train, seed=seed, batch_size=8)

train_generator = zip(x_generator, y_generator)

In [ ]:
model = unet_model(input_shape=(NEW_SIZE, NEW_SIZE, 3), initial_filters=4)
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=50, epochs=3)

In [ ]:
model.save('my_model')